In [1]:
# Importar las librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from imblearn.over_sampling import SMOTE

# Cargar el dataset
df = pd.read_csv('../data_en/spam_ham_dataset.csv')

# Visualizar las primeras filas del dataset
print("Primeras filas del dataset:")
print(df.head())



Primeras filas del dataset:
   Unnamed: 0 label                                               text  \
0         605   ham  Subject: enron methanol ; meter # : 988291\r\n...   
1        2349   ham  Subject: hpl nom for january 9 , 2001\r\n( see...   
2        3624   ham  Subject: neon retreat\r\nho ho ho , we ' re ar...   
3        4685  spam  Subject: photoshop , windows , office . cheap ...   
4        2030   ham  Subject: re : indian springs\r\nthis deal is t...   

   label_num  
0          0  
1          0  
2          0  
3          1  
4          0  


In [2]:
# Verificar y eliminar duplicados
print("\nNúmero de filas antes de eliminar duplicados:", df.shape[0])
df.drop_duplicates(subset='text', keep='first', inplace=True)
print("Número de filas después de eliminar duplicados:", df.shape[0])




Número de filas antes de eliminar duplicados: 5171
Número de filas después de eliminar duplicados: 4993


In [3]:
# Resumen estadístico del dataset
print("\nResumen estadístico:")
print(df.describe())

# Verificar datos faltantes
print("\nDatos faltantes en cada columna:")
print(df.isnull().sum())


Resumen estadístico:
        Unnamed: 0    label_num
count  4993.000000  4993.000000
mean   2581.528139     0.292810
std    1505.740921     0.455098
min       0.000000     0.000000
25%    1251.000000     0.000000
50%    2578.000000     0.000000
75%    3886.000000     1.000000
max    5170.000000     1.000000

Datos faltantes en cada columna:
Unnamed: 0    0
label         0
text          0
label_num     0
dtype: int64


In [4]:
df.head(20)

,Unnamed: 0,label,text,label_num
0,605,ham,Subject: enron methanol ; meter # : 988291\r\n...,0
1,2349,ham,"Subject: hpl nom for january 9 , 2001\r\n( see...",0
2,3624,ham,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0
3,4685,spam,"Subject: photoshop , windows , office . cheap ...",1
4,2030,ham,Subject: re : indian springs\r\nthis deal is t...,0
5,2949,ham,Subject: ehronline web address change\r\nthis ...,0
6,2793,ham,Subject: spring savings certificate - take 30 ...,0
7,4185,spam,Subject: looking for medication ? we ` re the ...,1
8,2641,ham,Subject: noms / actual flow for 2 / 26\r\nwe a...,0
9,1870,ham,"Subject: nominations for oct . 21 - 23 , 2000\...",0


In [5]:
# Eliminar 'Subject: ' de cada mensaje de texto
df['text'] = df['text'].str.replace('Subject: ', '')

# Visualizar las primeras filas del dataset después de la modificación
print("\nPrimeras filas del dataset después de eliminar 'Subject: ':")
print(df.head())

# Guardar el dataset modificado si es necesario
df.to_csv('../data_en/spam_ham_dataset_modificado.csv', index=False)


Primeras filas del dataset después de eliminar 'Subject: ':
   Unnamed: 0 label                                               text  \
0         605   ham  enron methanol ; meter # : 988291\r\nthis is a...   
1        2349   ham  hpl nom for january 9 , 2001\r\n( see attached...   
2        3624   ham  neon retreat\r\nho ho ho , we ' re around to t...   
3        4685  spam  photoshop , windows , office . cheap . main tr...   
4        2030   ham  re : indian springs\r\nthis deal is to book th...   

   label_num  
0          0  
1          0  
2          0  
3          1  
4          0  


In [6]:

# Cargar el dataset modificado
df_modificado = pd.read_csv('../data_en/spam_ham_dataset_modificado.csv')

# Visualizar las primeras filas para verificar
print("Primeras filas del dataset modificado:")
df_modificado.head(20)



Primeras filas del dataset modificado:


,Unnamed: 0,label,text,label_num
0,605,ham,enron methanol ; meter # : 988291\r\nthis is a...,0
1,2349,ham,"hpl nom for january 9 , 2001\r\n( see attached...",0
2,3624,ham,"neon retreat\r\nho ho ho , we ' re around to t...",0
3,4685,spam,"photoshop , windows , office . cheap . main tr...",1
4,2030,ham,re : indian springs\r\nthis deal is to book th...,0
5,2949,ham,ehronline web address change\r\nthis message i...,0
6,2793,ham,spring savings certificate - take 30 % off\r\n...,0
7,4185,spam,looking for medication ? we ` re the best sour...,1
8,2641,ham,noms / actual flow for 2 / 26\r\nwe agree\r\n-...,0
9,1870,ham,"nominations for oct . 21 - 23 , 2000\r\n( see ...",0


In [7]:
# Dividir el dataset en conjunto de entrenamiento, validación y prueba
# Primero, dividir en entrenamiento y prueba (80% entrenamiento, 20% prueba)
X_train, X_test, y_train, y_test = train_test_split(df_modificado['text'], df_modificado['label_num'], test_size=0.2, random_state=42, stratify=df_modificado['label_num'])

# Luego, dividir el conjunto de entrenamiento en entrenamiento y validación (75% entrenamiento, 25% validación)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42, stratify=y_train)

# Mostrar tamaños de los conjuntos resultantes
print("Tamaño del conjunto de entrenamiento:", X_train.shape[0])
print("Tamaño del conjunto de validación:", X_val.shape[0])
print("Tamaño del conjunto de prueba:", X_test.shape[0])

Tamaño del conjunto de entrenamiento: 2995
Tamaño del conjunto de validación: 999
Tamaño del conjunto de prueba: 999


In [8]:
# Calcular y mostrar la distribución de 'ham' y 'spam' en cada conjunto
def calcular_distribucion(y):
    ham_count = (y == 0).sum()
    spam_count = (y == 1).sum()
    total = len(y)
    return ham_count, spam_count, total

ham_train, spam_train, total_train = calcular_distribucion(y_train)
ham_val, spam_val, total_val = calcular_distribucion(y_val)
ham_test, spam_test, total_test = calcular_distribucion(y_test)

print("\nDistribución en el conjunto de entrenamiento:")
print(f"Ham: {ham_train} ({(ham_train/total_train)*100:.2f}%)")
print(f"Spam: {spam_train} ({(spam_train/total_train)*100:.2f}%)")

print("\nDistribución en el conjunto de validación:")
print(f"Ham: {ham_val} ({(ham_val/total_val)*100:.2f}%)")
print(f"Spam: {spam_val} ({(spam_val/total_val)*100:.2f}%)")

print("\nDistribución en el conjunto de prueba:")
print(f"Ham: {ham_test} ({(ham_test/total_test)*100:.2f}%)")
print(f"Spam: {spam_test} ({(spam_test/total_test)*100:.2f}%)")


Distribución en el conjunto de entrenamiento:
Ham: 2118 (70.72%)
Spam: 877 (29.28%)

Distribución en el conjunto de validación:
Ham: 707 (70.77%)
Spam: 292 (29.23%)

Distribución en el conjunto de prueba:
Ham: 706 (70.67%)
Spam: 293 (29.33%)


In [9]:
# Guardar los conjuntos en archivos separados
X_train.to_csv('../data_en/train.csv', index=False)
X_val.to_csv('../data_en/validation.csv', index=False)
X_test.to_csv('../data_en/test.csv', index=False)

In [10]:


# Vectorización del texto usando TF-IDF
vectorizer = TfidfVectorizer(stop_words='english')
X_train_tfidf = vectorizer.fit_transform(X_train)
X_val_tfidf = vectorizer.transform(X_val)
X_test_tfidf = vectorizer.transform(X_test)

# Manejo del desbalanceo de clases usando SMOTE solo en el conjunto de entrenamiento
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train_tfidf, y_train)

# Crear un modelo de Naive Bayes
model = MultinomialNB()

# Entrenar el modelo con datos sobremuestreados
model.fit(X_train_res, y_train_res)

# Hacer predicciones en el conjunto de validación
y_val_pred = model.predict(X_val_tfidf)

# Evaluar el modelo en el conjunto de validación
print("Reporte de Clasificación en el conjunto de validación:")
print(classification_report(y_val, y_val_pred))

print("Matriz de Confusión en el conjunto de validación:")
print(confusion_matrix(y_val, y_val_pred))

print("Exactitud en el conjunto de validación:")
print(accuracy_score(y_val, y_val_pred))

# Hacer predicciones en el conjunto de prueba
y_test_pred = model.predict(X_test_tfidf)

# Evaluar el modelo en el conjunto de prueba
print("\nReporte de Clasificación en el conjunto de prueba:")
print(classification_report(y_test, y_test_pred))

print("Matriz de Confusión en el conjunto de prueba:")
print(confusion_matrix(y_test, y_test_pred))

print("Exactitud en el conjunto de prueba:")
print(accuracy_score(y_test, y_test_pred))

Reporte de Clasificación en el conjunto de validación:
              precision    recall  f1-score   support

           0       1.00      0.97      0.98       707
           1       0.93      0.99      0.96       292

    accuracy                           0.98       999
   macro avg       0.96      0.98      0.97       999
weighted avg       0.98      0.98      0.98       999

Matriz de Confusión en el conjunto de validación:
[[686  21]
 [  2 290]]
Exactitud en el conjunto de validación:
0.9769769769769769

Reporte de Clasificación en el conjunto de prueba:
              precision    recall  f1-score   support

           0       0.99      0.95      0.97       706
           1       0.90      0.99      0.94       293

    accuracy                           0.96       999
   macro avg       0.95      0.97      0.96       999
weighted avg       0.97      0.96      0.96       999

Matriz de Confusión en el conjunto de prueba:
[[674  32]
 [  4 289]]
Exactitud en el conjunto de prueba:
0.

Los resultados que has obtenido muestran el rendimiento del modelo de clasificación (en este caso, Naive Bayes) en los conjuntos de validación y prueba. Aquí te explico qué significan las métricas del reporte de clasificación y de la matriz de confusión:

### Conjunto de Validación:

**Reporte de Clasificación:**
- **Precision**: Mide la proporción de mensajes etiquetados como spam que realmente son spam. Para la clase "ham" (0), es del 100%, y para la clase "spam" (1), es del 93%.
- **Recall**: Mide la proporción de mensajes que realmente son spam que fueron identificados correctamente como spam por el modelo. Para la clase "ham" es del 97%, y para la clase "spam" es del 99%.
- **F1-score**: Es una medida balanceada entre precision y recall. Para la clase "ham" es del 98%, y para la clase "spam" es del 96%.
- **Support**: Es el número de ocurrencias de cada clase en el conjunto de validación.

**Matriz de Confusión:**
- La matriz de confusión muestra la distribución de las predicciones del modelo comparadas con las etiquetas reales. 
- En tu caso, la clase "ham" (0) tiene 686 predicciones correctas y 21 falsos positivos. La clase "spam" (1) tiene 290 predicciones correctas y 2 falsos negativos.

**Exactitud (Accuracy)**: Es la proporción de predicciones correctas sobre el total de predicciones realizadas. En el conjunto de validación, la exactitud es del 97.7%.

### Conjunto de Prueba:

**Reporte de Clasificación:**
- Los valores son similares al conjunto de validación, pero representan el rendimiento del modelo en datos completamente nuevos que no se usaron durante el entrenamiento ni la validación.

**Matriz de Confusión:**
- La clase "ham" (0) tiene 674 predicciones correctas y 32 falsos positivos. La clase "spam" (1) tiene 289 predicciones correctas y 4 falsos negativos.

**Exactitud (Accuracy)**: Es del 96.4%, lo que indica que el modelo tiene un buen rendimiento general en la clasificación de mensajes de texto como "ham" o "spam" en el conjunto de prueba.

### Interpretación:

- El modelo muestra un rendimiento sólido tanto en el conjunto de validación como en el conjunto de prueba, con altas precisiones y recalls para ambas clases.
- La diferencia entre el rendimiento en el conjunto de validación y el conjunto de prueba es mínima, lo cual indica que el modelo generaliza bien a nuevos datos.
- Es importante seguir monitoreando el rendimiento del modelo y considerar ajustes adicionales según sea necesario, como la optimización de hiperparámetros o el uso de técnicas avanzadas de procesamiento de texto si es necesario mejorar aún más el rendimiento.

En resumen, los resultados obtenidos son muy prometedores y reflejan un modelo eficaz para la clasificación de mensajes de texto en "ham" y "spam".

Entrenamos el modelo con todo train y validación

In [11]:
# Importar las librerías necesarias
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from imblearn.over_sampling import SMOTE

# Cargar el dataset modificado
df_modificado = pd.read_csv('../data_en/spam_ham_dataset_modificado.csv')

# Combinar entrenamiento y validación
X_train_val = df_modificado['text']
y_train_val = df_modificado['label_num']

# Vectorización del texto usando TF-IDF
vectorizer = TfidfVectorizer(stop_words='english')
X_train_val_tfidf = vectorizer.fit_transform(X_train_val)

# Manejo del desbalanceo de clases usando SMOTE
smote = SMOTE(random_state=42)
X_train_val_res, y_train_val_res = smote.fit_resample(X_train_val_tfidf, y_train_val)

# Crear un modelo de Naive Bayes
model = MultinomialNB()

# Entrenar el modelo con todos los datos de entrenamiento y validación
model.fit(X_train_val_res, y_train_val_res)




MultinomialNB()

In [12]:
import joblib
# Guardar el modelo y el vectorizador
joblib.dump(model, '../modelos_y_vectorizadores_en/naive_bayes_model.pkl')
joblib.dump(vectorizer, '../modelos_y_vectorizadores_en/tfidf_vectorizer.pkl')

print("Modelo y vectorizador guardados correctamente.")

Modelo y vectorizador guardados correctamente.


In [17]:
'''# Cargar el conjunto de datos de prueba (test.csv)
test_df = pd.read_csv('../data_en/test.csv')

# Verificar las primeras filas para entender la estructura del archivo
print(test_df.head())'''

                                                text
0  frontrea plant , gas\r\nwhen : wednesday , jul...
1  news\r\nat least two bids seen for enron tradi...
2  quailty web solutions , involutory\r\noverton ...
3  txu fuels / sds nomination for december 2000\r...
4  use the links to get more information on rx me...


In [18]:
'''# Cargar el conjunto de datos de prueba (train.csv)
train_df = pd.read_csv('../data_en/train.csv')

# Verificar las primeras filas para entender la estructura del archivo
print(train_df.head())'''

                                                text
0  sitara availability - status update\r\nwe are ...
1  hub park and lend agreement\r\ndaren ,\r\nwe a...
2  new message .\r\nyou have [ 1 ] new message .\...
3  congratulations ! ! ! you have won .\r\nnation...
4  last check\r\ndarren ,\r\nplease review the na...
